In [44]:

import pandas as pd
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
import tweepy
import json
import os
from collections import Counter
import numpy as np
from collections import defaultdict
from matplotlib import pyplot as plt
import time
from datetime import datetime
from dateutil.parser import parse

In [45]:
# load tweets.json file
def load_json(file_name):
    result=[]
    with open(file_name, 'r') as fp:
        line=fp.readline()
        while line:
            tjson=json.loads(line) #decode json
            result.append({
                 "Date":tjson["created_at"],
                "id":tjson["id_str"],
                 "user_name":tjson["user"]["name"],
                   
                    "favorite_count":tjson["favorite_count"], #number of favorites
                    "retweet_count":tjson["retweet_count"], #number of retweets
                    "user_id_str":tjson["user"]["id_str"],
                
                
            })

            line=fp.readline()
    return result

# load tweets.json file
def load_old_json(file_name):
    result=[]
    with open(file_name, 'r') as fp:
        line=fp.readline()
        while line:
            tjson=json.loads(line) #decode json
            result.append({
                "Date":tjson["Date"],
                "id":tjson["id"],
                "favorite_count":tjson["favorite_count"], #number of favorites
                "retweet_count":tjson["retweet_count"], #number of retweets
                "text":tjson["text"]
            })
            line=fp.readline()
    return result


In [46]:
#data_list=load_json("tweets.json")
data_list=load_old_json("old_tweets.json")
df=pd.DataFrame(data=data_list)
print('done')
#df = df.drop_duplicates(subset=['id'], keep=False)

#converting date into more readable form 
Date_list= df["Date"].values
dates = []
for date in Date_list:
    dates.append(parse(date).date())
df["Date"]=dates
print(df)

done
              Date                   id  favorite_count  retweet_count  \
0       2020-01-22  1220134040705142789              18             17   
1       2020-01-22  1220134028906463235               0              1   
2       2020-01-22  1220133978046259200               3              2   
3       2020-01-22  1220133958094151680               0              0   
4       2020-01-22  1220133790346956800               3              0   
...            ...                  ...             ...            ...   
271112  2020-04-02  1245501262226915334               0              0   
271113  2020-04-02  1245501257780994055               3              4   
271114  2020-04-02  1245501252018024454               4              3   
271115  2020-04-02  1245501201552150531               2              0   
271116  2020-04-02  1245501196518879233               2              2   

                                                     text  
0       AGW GREEN WARRIORS CENSOR 1st Amendmen

In [ ]:
### Sentiment Analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

SIA = SentimentIntensityAnalyzer()
sentiments = []
for t in df["text"].values:
    sentiment_dict = SIA.polarity_scores(t)
    
    # decide sentiment as positive, negative and neutral 
    if sentiment_dict['compound'] >= 0.05 : 
        sentiments.append("Positive") 
        
    elif sentiment_dict['compound'] <= - 0.05 : 
        sentiments.append("Negative") 
    else :
        sentiments.append("Neutral")
        
df["sentiment"]= sentiments

def count_pos(series):
    return len([x for x in series if x=="Positive"])
def count_neg(series):
    return len([x for x in series if x=="Negative"])
def count_nue(series):
    return len([x for x in series if x=="Neutral"])

df.to_csv("data_with_sent.csv", index=True)


In [51]:

#groupby date
group=df.groupby(by=["Date"])
new_df=group.aggregate({
    "id":["count"],
    "favorite_count":["sum"],
    "retweet_count":["sum"],
    "sentiment":[count_pos, count_neg, count_nue]
})
new_df.rename(columns={"id_str":"tweet_count"}, inplace=True)
print(new_df)
new_df.to_csv("old_climate_data.csv", index=True)


              id favorite_count retweet_count sentiment                    
           count            sum           sum count_pos count_neg count_nue
Date                                                                       
2020-01-22  9504          49563         20441      5207      1928      2369
2020-01-23  6427          43605         16548      2674      1744      2009
2020-01-24  5840          46585         16407      2744      1446      1650
2020-01-25  4194          39124         14920      2063       970      1161
2020-01-26  3398          16882          7865      1472       850      1076
...          ...            ...           ...       ...       ...       ...
2020-03-28  1911          14566          5240       791       495       625
2020-03-29  1585           7182          4709       556       502       527
2020-03-30  2216           9481          4514       909       642       665
2020-04-01  2470          12600          5783      1024       745       701
2020-04-02  